# ST446 Distributed Computing for Big Data


## Week 4 class: Spark SQL and DataFrames


### LT 2021

This notebook contains code samples from Chapter 4 of the book Learning PySpark.

## 1. Generate a DataFrame 

### a. Data from file
We again use the `author-large.txt` file from dblp, which we used in previous exercises. We load the data into a DataFrame using the function `spark.read.csv()`.

(Please create a folder called `flight-data` in your bucket and copy the files under `week03/class/flight-data/` into your bucket. And the result should be like

```
(base) LSE021353-2:~ st446$ gsutil ls gs://jialin-bucket/flight-data
gs://jialin-bucket/flight-data/airport-codes-na.txt
gs://jialin-bucket/flight-data/departuredelays.csv
```

In [1]:
filename = 'gs://jialin-bucket/data/author-large.txt'

author_large_no_schema = spark.read.csv(filename, 
                    header='false', sep='\t')
author_large_no_schema.createOrReplaceTempView("author_large_no_schema")

#### Inferring the Schema Using Reflection
Spark can infer the schema using _reflection_; i.e. automaticaly determining the schema of the data based on sampling the data.

In [2]:
author_large_no_schema.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- _c1: string (nullable = true)
 |-- _c2: string (nullable = true)
 |-- _c3: string (nullable = true)



#### Programmatically Specifying the Schema
Spark also allows to programmatically specify the schema.

In [3]:
from pyspark.sql.types import *

schema = StructType([
    StructField("author", StringType(), True),    
    StructField("journal", StringType(), True),
    StructField("title", StringType(), True),
    StructField("year", LongType(), True)
])

author_large = spark.read.csv(filename, 
                    header='false', schema=schema, sep='\t')
author_large.createOrReplaceTempView("author_large")

In [4]:
author_large.printSchema()

root
 |-- author: string (nullable = true)
 |-- journal: string (nullable = true)
 |-- title: string (nullable = true)
 |-- year: long (nullable = true)



As you can see from above, we can programmatically apply the `schema` instead of allowing the Spark to infer the schema via reflection.

Additional resources:
* [PySpark API Reference](https://spark.apache.org/docs/2.0.0/api/python/pyspark.sql.html)
* [Spark SQL, DataFrames, and Datasets Guide](https://spark.apache.org/docs/latest/sql-programming-guide.html#programmatically-specifying-the-schema) (for programmatically specifying the schema from a `csv` file).

#### SparkSession

Notice that we're no longer using `sqlContext.read...` but instead `spark.read...`.  This is because as part of Spark 2.0, `HiveContext`, `SQLContext`, `StreamingContext`, `SparkContext` have been merged together into the Spark Session `spark`, which provides:
* entry point for reading data,
* working with metadata,
* configuration,
* cluster resource management.

For more information, see [How to use SparkSession in Apache Spark 2.0](http://bit.ly/2br0Fr1).

### b. Generate your own DataFrame
Instead of accessing the file system, let's create a DataFrame by generating the data.  In this case, we'll first create the `stringRDD` RDD and then convert it into a DataFrame when we're reading `stringJSONRDD` using `spark.read.json`.

In [5]:
# Generate our own JSON data 
#   This way we don't have to access the file system yet.
stringJSONRDD = sc.parallelize((""" 
  { "id": "123",
    "name": "Katie",
    "age": 19,
    "eyeColor": "brown"
  }""",
   """{
    "id": "234",
    "name": "Michael",
    "age": 22,
    "eyeColor": "green"
  }""", 
  """{
    "id": "345",
    "name": "Simone",
    "age": 23,
    "eyeColor": "blue"
  }""")
) 
# Create DataFrame
swimmersJSON = spark.read.json(stringJSONRDD) 
# Create temporary table
swimmersJSON.createOrReplaceTempView("swimmersJSON") 
# DataFrame API
swimmersJSON.show()

+---+--------+---+-------+
|age|eyeColor| id|   name|
+---+--------+---+-------+
| 19|   brown|123|  Katie|
| 22|   green|234|Michael|
| 23|    blue|345| Simone|
+---+--------+---+-------+



## 2. Querying with the DataFrame API

Spark allows you to query a DataFrame using the DataFrame API.

### a. Show first few rows

In [6]:
author_large.show(10)

+--------------------+--------------------+--------------------+----+
|              author|             journal|               title|year|
+--------------------+--------------------+--------------------+----+
|   Jurgen Annevelink|Modern Database S...|Object SQL - A La...|1995|
|         Rafiul Ahad|Modern Database S...|Object SQL - A La...|1995|
|      Amelia Carlson|Modern Database S...|Object SQL - A La...|1995|
|   Daniel H. Fishman|Modern Database S...|Object SQL - A La...|1995|
|  Michael L. Heytens|Modern Database S...|Object SQL - A La...|1995|
|        William Kent|Modern Database S...|Object SQL - A La...|1995|
|     Jos A. Blakeley|Modern Database S...|OQL[C++]: Extendi...|1995|
|      Yuri Breitbart|Modern Database S...|Transaction Manag...|1995|
|Hector Garcia-Molina|Modern Database S...|Transaction Manag...|1995|
|Abraham Silberschatz|Modern Database S...|Transaction Manag...|1995|
+--------------------+--------------------+--------------------+----+
only showing top 10 

### b. Count rows

In [7]:
author_large.count()

2225370

### c. Filter 

Here we get the titles with year 2000:

In [8]:
author_large.select(author_large.title).distinct().filter(author_large.year == 2000).show(10)

+--------------------+
|               title|
+--------------------+
|Leveraging Relati...|
|Faster FFTs via A...|
|From Causal Theor...|
|Team-Aware Multir...|
|Key Management fo...|
|An Algorithm for ...|
|Belief Revision P...|
|ATM network manag...|
|Building Virtual ...|
|A Modular Approac...|
+--------------------+
only showing top 10 rows



In [9]:
author_large.select("title").distinct().filter("year = 2000").show(10)

+--------------------+
|               title|
+--------------------+
|Leveraging Relati...|
|Faster FFTs via A...|
|From Causal Theor...|
|Team-Aware Multir...|
|Key Management fo...|
|An Algorithm for ...|
|Belief Revision P...|
|ATM network manag...|
|Building Virtual ...|
|A Modular Approac...|
+--------------------+
only showing top 10 rows



## 3. Querying with Spark SQL
You can also write your queries using `Spark SQL` - a SQL dialect that is compatible with the Hive Query Language (or HiveQL). The following codes produces the same output that in the section "Querying with the DataFrame API".

### a. Show first few rows

In [10]:
spark.sql("select * from author_large limit 10 ").show()

+--------------------+--------------------+--------------------+----+
|              author|             journal|               title|year|
+--------------------+--------------------+--------------------+----+
|   Jurgen Annevelink|Modern Database S...|Object SQL - A La...|1995|
|         Rafiul Ahad|Modern Database S...|Object SQL - A La...|1995|
|      Amelia Carlson|Modern Database S...|Object SQL - A La...|1995|
|   Daniel H. Fishman|Modern Database S...|Object SQL - A La...|1995|
|  Michael L. Heytens|Modern Database S...|Object SQL - A La...|1995|
|        William Kent|Modern Database S...|Object SQL - A La...|1995|
|     Jos A. Blakeley|Modern Database S...|OQL[C++]: Extendi...|1995|
|      Yuri Breitbart|Modern Database S...|Transaction Manag...|1995|
|Hector Garcia-Molina|Modern Database S...|Transaction Manag...|1995|
|Abraham Silberschatz|Modern Database S...|Transaction Manag...|1995|
+--------------------+--------------------+--------------------+----+



### b. Count rows

In [11]:
 spark.sql("select count(1) from author_large").show()

+--------+
|count(1)|
+--------+
| 2225370|
+--------+



### c. Filter 

Here we get the titles with year 2000:

In [12]:
spark.sql("select DISTINCT title from author_large where year = 2000 limit 10").show()

+--------------------+
|               title|
+--------------------+
|Leveraging Relati...|
|Faster FFTs via A...|
|From Causal Theor...|
|Team-Aware Multir...|
|Key Management fo...|
|An Algorithm for ...|
|Belief Revision P...|
|ATM network manag...|
|Building Virtual ...|
|A BIST methodolog...|
+--------------------+



Here we query publications with title starting with letter `b`: 

In [13]:
spark.sql("select title from author_large where title like 'b%' limit 10").show()

+--------------------+
|               title|
+--------------------+
|      buyer's agent.|
|      buyer's agent.|
|bicACO: An Ant Co...|
|bicACO: An Ant Co...|
|bicACO: An Ant Co...|
|            bottom).|
|ber den Abstrakti...|
|berlegungen beim ...|
|ber die Realisier...|
|berlegungen zu ei...|
+--------------------+



## 4. Query by joining 2 tables

Let's run a flight performance using DataFrames; let's first build the DataFrames from the source datasets.

In [14]:
# Set File Paths
flightPerfFilePath = "gs://jialin-bucket/flight-data/departuredelays.csv"
airportsFilePath = "gs://jialin-bucket/flight-data/airport-codes-na.txt"

# Obtain Airports dataset
airports = spark.read.csv(airportsFilePath, header='true', inferSchema='true', sep='\t')
airports.createOrReplaceTempView("airports")

# Obtain Departure Delays dataset
flightPerf = spark.read.csv(flightPerfFilePath, header='true')
flightPerf.createOrReplaceTempView("FlightPerformance")

# Cache the Departure Delays dataset 
flightPerf.cache()

DataFrame[date: string, delay: string, distance: string, origin: string, destination: string]

In [15]:
airports.show(5)
flightPerf.show(5)

+----------+-----+-------+----+
|      City|State|Country|IATA|
+----------+-----+-------+----+
|Abbotsford|   BC| Canada| YXX|
|  Aberdeen|   SD|    USA| ABR|
|   Abilene|   TX|    USA| ABI|
|     Akron|   OH|    USA| CAK|
|   Alamosa|   CO|    USA| ALS|
+----------+-----+-------+----+
only showing top 5 rows

+--------+-----+--------+------+-----------+
|    date|delay|distance|origin|destination|
+--------+-----+--------+------+-----------+
|01011245|    6|     602|   ABE|        ATL|
|01020600|   -8|     369|   ABE|        DTW|
|01021245|   -2|     602|   ABE|        ATL|
|01020605|   -4|     602|   ABE|        ATL|
|01031245|   -4|     602|   ABE|        ATL|
+--------+-----+--------+------+-----------+
only showing top 5 rows



Now we query flight departure delays by cities in WA by joining the performance and airport tables with the airport codes (to identify state and city):

In [16]:
# Query Sum of Flight Delays by City and Origin Code (for Washington State)
spark.sql("select a.City, f.origin, sum(f.delay) as Delays from FlightPerformance f join airports a on a.IATA = f.origin where a.State = 'WA' group by a.City, f.origin order by sum(f.delay) desc").show()

+-------+------+--------+
|   City|origin|  Delays|
+-------+------+--------+
|Seattle|   SEA|159086.0|
|Spokane|   GEG| 12404.0|
|  Pasco|   PSC|   949.0|
+-------+------+--------+



Here we query flight departure delays by States in the US:

In [17]:
# Query Sum of Flight Delays by State (for the US)
spark.sql("select a.State, sum(f.delay) as Delays from FlightPerformance f join airports a on a.IATA = f.origin where a.Country = 'USA' group by a.State ").show()

+-----+---------+
|State|   Delays|
+-----+---------+
|   SC|  80666.0|
|   AZ| 401793.0|
|   LA| 199136.0|
|   MN| 256811.0|
|   NJ| 452791.0|
|   OR| 109333.0|
|   VA|  98016.0|
| null| 397237.0|
|   RI|  30760.0|
|   WY|  15365.0|
|   KY|  61156.0|
|   NH|  20474.0|
|   MI| 366486.0|
|   NV| 474208.0|
|   WI| 152311.0|
|   ID|  22932.0|
|   CA|1891919.0|
|   CT|  54662.0|
|   NE|  59376.0|
|   MT|  19271.0|
+-----+---------+
only showing top 20 rows



For more information, please refer to:
* [Spark SQL, DataFrames and Datasets Guide](http://spark.apache.org/docs/latest/sql-programming-guide.html#sql)
* [PySpark SQL Module: DataFrame](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame)
* [PySpark SQL Functions Module](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#module-pyspark.sql.functions)